<a href="https://colab.research.google.com/github/sucharithasu/Secure-Image-Retrieval/blob/main/AES_Encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.3 MB/s eta 0:00:00


In [12]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from PIL import Image
import numpy as np

# Function to read image and convert to bytes
def image_to_bytes(image_path):
    img = Image.open(image_path).convert('L')  # Grayscale
    img_array = np.array(img)
    img_bytes = img_array.tobytes()
    return img_bytes, img.size, img.mode

# Function to encrypt image bytes using AES
def encrypt_image_bytes(image_bytes, key):
    cipher = AES.new(key, AES.MODE_ECB)  # Using ECB for simplicity
    padded_data = pad(image_bytes, AES.block_size)  # Pad to 16-byte multiple
    encrypted_bytes = cipher.encrypt(padded_data)
    return encrypted_bytes

# Function to convert bytes back to image
def bytes_to_image(encrypted_bytes, size, mode):
    encrypted_array = np.frombuffer(encrypted_bytes, dtype=np.uint8)
    encrypted_array = encrypted_array[:size[0] * size[1]]  # Trim any padding
    encrypted_image = encrypted_array.reshape((size[1], size[0]))  # Note: size is (width, height)
    return Image.fromarray(encrypted_image, mode='L')

# Main execution
if __name__ == "__main__":
    image_path = '/content/00024743_000.png'  # Replace with your image path
    key = b'ThisIsASecretKey'  # 16 bytes for AES-128

    img_bytes, size, mode = image_to_bytes(image_path)
    encrypted_bytes = encrypt_image_bytes(img_bytes, key)
    encrypted_image = bytes_to_image(encrypted_bytes, size, mode)

    encrypted_image.save('encrypted_image4.png')
    print("Image encrypted and saved as 'encrypted_image.png'")
    encrypted_image.show()


Image encrypted and saved as 'encrypted_image.png'


In [25]:
from PIL import Image
import os
import math

def combine_images(folder_path, grid_size=(1, 4), image_size=(128, 128)):
    # List all image files
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png'))]

    total_images = len(image_files)
    rows, cols = grid_size

    # Adjust grid size if needed
    if total_images < rows * cols:
        cols = math.ceil(total_images / rows)

    # Create a blank canvas for the final image
    combined_width = cols * image_size[0]
    combined_height = rows * image_size[1]
    combined_image = Image.new('RGB', (combined_width, combined_height), color='white')

    for idx, file in enumerate(image_files):
        if idx >= rows * cols:
            break
        img_path = os.path.join(folder_path, file)
        img = Image.open(img_path).convert('RGB')
        img = img.resize(image_size)

        x = (idx % cols) * image_size[0]
        y = (idx // cols) * image_size[1]
        combined_image.paste(img, (x, y))

    return combined_image

# Example usage
folder = '/content/Encrypted_Images'  # Replace with your folder path
result_image = combine_images(folder, grid_size=(1, 4), image_size=(128, 128))
result_image.show()
result_image.save('combined_output1.png')
